In [1]:
import requests
from lxml import etree
from time import sleep
import re
import pandas as pd
from pandas import DataFrame, Series
 
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.8',
    'Cache-Control': 'max-age=0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36',
    'Connection': 'keep-alive',
    'Referer': 'http://www.baidu.com/'
}

In [2]:
# 获取爬取页面列表
def get_pages(url):
    second_pages = []
    third_pages = []
    html_list = []
    res = requests.get(url=url,headers=headers)
    pat = re.compile(r'/list[0-9A-Z]')
    ls = pat.findall(res.text)
    for next_url in ls:
        second_pages.append(url + next_url)
#     print(second_pages)
    
    for page in second_pages:
        res = requests.get(url=page,headers=headers)
        tree = etree.HTML(res.text)
        line_list = tree.xpath("//div[@class='list clearfix']/a/@href")
        for line in line_list:
            third_pages.append(url+line)
#     print(third_pages) 
    
    for page in third_pages:
        html = requests.get(url=page,headers=headers)
        sleep(0.5)
        html_list.append(html.text)
#     print(html_list)
    return html_list

In [3]:
# 对第三级页面进行爬取
def get_messages(html_list):
    messages = []
    for html in html_list:
        tree = etree.HTML(html)
        dic = {}
        try:
            dic["lineName"] = tree.xpath("//h1//text()")[0]
        except:
            dic["lineName"] = ''
        try:
            dic["time"] = tree.xpath("//ul[@class='bus-desc']//text()")[0]
        except:
            dic["time"] = ''
        try:
            dic["price"] = tree.xpath("//ul[@class='bus-desc']//text()")[1]
        except:
            dic["price"] = ''
        try:
            dic["campony"] = tree.xpath("//ul[@class='bus-desc']//a//text()")[0]
        except:
            dic["campony"] = ''
        
        try:
            lines = tree.xpath("//div[@class='bus-lzlist mb15']")
        except:
            dic["upline"] = ''
            dic['downline'] = ''

        # 获取上行线路    
        ls = lines[0].xpath(".//text()")
        dic["upline"] = ls
        
        # 获取下行线路
        if len(lines) > 1:
            ls = lines[1].xpath(".//text()")
            dic["downline"] = ls
        messages.append(dic)
    return messages

In [4]:
# 保存爬取数据
def save_messages(messages):
    df = DataFrame(messages,columns=["lineName", "time", "price", "campony", "upline", "downline"])
    print(df.head)
    df.to_csv("bus_messages.csv", encoding='utf_8_sig')

In [5]:
if __name__ == '__main__':
    url = "https://yichang.8684.cn/"
    html_list = get_pages(url)
    messages = get_messages(html_list)
    save_messages(messages)

<bound method NDFrame.head of          lineName                                               time  \
0      宜昌11路公交车路线                运行时间：移动通信大楼6:00-21:00|四方堰6:34-21:34   
1      宜昌12路公交车路线             运行时间：夷陵区市民服务中心5:50-21:00|步行街6:33-21:43   
2     宜昌108路公交车路线  运行时间：长江市场客运站（夏）5:30-18:05 （冬）5:40-17:35|鸦鹊岭客运站...   
3     宜昌109路公交车路线  运行时间：疫情期间交运夷陵客运站6:30-17:30(交运夷陵客运站5:30-18:50|鸦...   
4     宜昌119路公交车路线            运行时间：市警示基地6:00-21:00|青岛路城东大道口6:30-21:30   
..            ...                                                ...   
161   宜昌枝江8路公交车路线              运行时间：余家溪轮渡码头5:55-20:10|枝江北站6:25-21:00   
162   宜昌枝江9路公交车路线                 运行时间：飞鹰码头5:55-19:30|枝江北站6:25-20:15   
163  宜昌枝江10路公交车路线  运行时间：英郡年华6:00-19:30|枝江北站6:40-20:15(弥陀寺7:45-17:50)   
164  宜昌枝江11路公交车路线              运行时间：三宁职校6:00-23:00|姚家港公汽首末站6:30-0:20   
165  宜昌枝江12路公交车路线                运行时间：飞鹰码头5:55-17:45|步步升景区6:45-18:30   

                                             price      campony  \
0    参考票价：票价2元 宜昌公交IC卡8折 交联卡、银联卡、宜知行、支